# Workflow

In this chapter, we are going to wrap our workflow into a re-usable function, see how we can handle and export the extracted tabular data and finally export all these tables when we analyze all images.

We can summarize our complete workflow as the following function which takes a three channel image as input:

In [1]:
import skimage
from microfilm.microplot import microshow
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi

In [2]:
def nuclei_in_out(image):
    image_nuclei = image[:,:,2]

    # filter image
    image_nuclei = skimage.filters.median(image_nuclei, skimage.morphology.disk(5))

    # create mask and clean-up
    mask_nuclei = image_nuclei > skimage.filters.threshold_otsu(image_nuclei)
    mask_nuclei = skimage.morphology.binary_closing(mask_nuclei, footprint=skimage.morphology.disk(5))
    mask_nuclei = ndi.binary_fill_holes(mask_nuclei, skimage.morphology.disk(5))

    labels_nuclei = skimage.morphology.label(mask_nuclei)

    mask_nuclei_eroded = skimage.morphology.binary_erosion(mask_nuclei, skimage.morphology.disk(10))

    labels_masked_inner = labels_nuclei * mask_nuclei_eroded
    labels_masked_outer = labels_nuclei * (1-mask_nuclei_eroded)

    measure_inner = skimage.measure.regionprops_table(
        label_image=labels_masked_inner, intensity_image=image[:,:,1],
        properties=('label', 'area', 'mean_intensity'))

    measure_outer = skimage.measure.regionprops_table(
        label_image=labels_masked_outer, intensity_image=image[:,:,1],
        properties=('label', 'area', 'mean_intensity'))
    
    return measure_inner, measure_outer


Let's verify that it works. We define here the image path as this is what we'll have in our complete analysis:

In [3]:
from pathlib import Path

image_path = Path('../data/cellatlas/19838_1252_F8_1.tiff')

In [4]:
image = skimage.io.imread(image_path)

In [5]:
m_in, m_out = nuclei_in_out(image)

In [6]:
m_in

{'label': array([ 1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 'area': array([ 5629.,  9904., 15070., 20884., 12972., 16068., 27912., 26131.,
        28071., 16176., 18853.]),
 'mean_intensity': array([28.21406999, 44.42982633, 53.1260783 , 49.79285577, 42.91111625,
        54.61090366, 52.34300659, 60.7661781 , 58.83042998, 54.78251731,
        60.94144168])}

In [7]:
m_out

{'label': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 'area': array([3675., 4484.,   52., 5746., 7425., 3176., 5261., 6906., 6183.,
        6505., 6131., 5290.]),
 'mean_intensity': array([19.60435374, 27.11239964, 31.88461538, 31.96293073, 31.77777778,
        28.91215365, 38.03592473, 37.74341153, 44.66876921, 41.29392775,
        26.7639863 , 38.31342155])}

## Comments on the function

Your are entirely free to define your functions as you like. For example, above we pass an image to the function, but we could as well pass a path and the image could be imported directly in the function. Also we return the two output dictionaries, but we could also directly save them in the function. Depending on your use case you may prefer one or the other solution.

## Generality

Once property you should generally aim for is *generality*. You might want to re-use this function for another project, and so you should be careful to not *hard-code* very specific features of your dataset in your code. For example you should avoid setting a fixed threshold value in your code. One solution for this is to pass such numbers as input to the function and to assign reasonable values.

## Handling tables: Pandas

Wherever we save the output dictionaries, we have to save them as tables. The most popular way of doing this in Python is to use the Pandas package which offers such a tabular data structure called a DataFrame. 

The first operation is to transform the dictionary into a DataFrame. Luckily this is very simple, as the keys from our dictionary can just be used to define the columns of our table:

In [8]:
import pandas as pd

out_df = pd.DataFrame(m_out)

In [9]:
out_df

,label,area,mean_intensity
0,1,3675.0,19.604354
1,2,4484.0,27.112400
2,3,52.0,31.884615
3,4,5746.0,31.962931
4,5,7425.0,31.777778
5,6,3176.0,28.912154
6,7,5261.0,38.035925
7,8,6906.0,37.743412
8,9,6183.0,44.668769
9,10,6505.0,41.293928


Now we see that this offers an actual table. Note though that it isn't an interactive table, i.e. you cannot point to lines and modify them.

These tables are very useful for post-processing as they allow combined multiple dataset, compute statistics on the table, group certain rows based on criteria etc. For example with

In [10]:
out_df.mean()

label                6.500000
area              5069.500000
mean_intensity      33.172806
dtype: float64

we can compute the mean of each column in the table without having to do that manually.

Pandas also offers a simple way to export such table into classical formats like csv. We only have to specify a valid path. For this we define a folder as a ```Path``` object:

In [11]:
from pathlib import Path
import os

export_dir = Path('../exports/')
if not export_dir.exists():
    os.makedirs(export_dir, exist_ok=True)
    

Now we want to keep track of which file this export corresponds to. So let's recover the name or ```stem``` of the image:

In [12]:
image_path.stem

'19838_1252_F8_1'

Now we can create a full path by also adding a csv extension and a suffix ```_out``` to specify that this corresponds to the membrane (outer part) intensity.

In [13]:
export_path = export_dir.joinpath(image_path.stem +'_out.csv')

Finally we can export the DataFrame or table using the ```to_csv``` method. We just us the additional options ```index=False``` to avoid storing a useless index in the csv file:

In [14]:
out_df.to_csv(export_path, index=False)

As a control, let's check the contents of the export folder:

In [15]:
list(export_dir.iterdir())

[PosixPath('../exports/27897_273_C8_2_in.csv'),
 PosixPath('../exports/24138_196_F7_2_in.csv'),
 PosixPath('../exports/19838_1252_F8_1_in.csv'),
 PosixPath('../exports/27897_273_C8_2_out.csv'),
 PosixPath('../exports/24138_196_F7_2_out.csv'),
 PosixPath('../exports/19838_1252_F8_1_out.csv')]

## Execute the workflow

We can finally run our full workflow. We just have to:
- define the list of files to analyze
- run the workflow on each file
- export the tables for each image

In [16]:
folder_to_analyze = Path('../data/cellatlas/')

In [17]:
files = list(folder_to_analyze.iterdir())
files

[PosixPath('../data/cellatlas/24138_196_F7_2.tiff'),
 PosixPath('../data/cellatlas/19838_1252_F8_1.tiff'),
 PosixPath('../data/cellatlas/_description.txt'),
 PosixPath('../data/cellatlas/27897_273_C8_2.tiff'),
 PosixPath('../data/cellatlas/.ipynb_checkpoints')]

In [18]:
export_folder = Path('../exports/')

for f in files:
    if f.suffix == '.tiff':
        
        image = skimage.io.imread(f)
        m_in, m_out = nuclei_in_out(image)
        
        m_in_df = pd.DataFrame(m_in)
        m_out_df = pd.DataFrame(m_out)
        
        export_in = export_folder.joinpath(f.stem+'_in.csv')
        export_out = export_folder.joinpath(f.stem+'_out.csv')
        
        m_in_df.to_csv(export_in, index=False)
        m_out_df.to_csv(export_out, index=False)